## Análisis de tipos de producto CvLAC

In [36]:
import pandas as pd
import requests
import time
import json
dtype={
'COD_RH':str,
'NRO_ID_GRUPO':str,
'COD_RH_MUNICIPIO':str,
'COD_INST':str,
'COD_INST_AVALA':str,
'COD_INST_MACRO':str,
'COD_INSTITUCION':str,         
'COD_INST_OTRO':str,
'ID_DEPARTAMENTO':str,
'COD_DEPA_DANE': str,
'COD_MUNI_DANE':str,
'COD_RH_OTRO':str,
'COD_RH_COAUTOR':str,
'COD_RH_CREA':str
 }

# Check some products

In [37]:
pf=pd.read_json('product_type_NIVEL_MAX.json',dtype=dtype)

In [3]:
JSON=False
if JSON:
    f=open('product_type_NIVEL_MAX.json','r')
    pf=json.load(f)
    f.close()

In [42]:
pf=pf.drop_duplicates(subset='COD_RH')

## Articles

In [4]:
producto='111: Publicado en revista especializada'
pf=pf[pf['NIVEL_MAX']==producto].reset_index(drop=True)

In [5]:
pf.shape

(65340, 5)

In [6]:
import getpass
RUN=False
if RUN:
    APIKEY=getpass.getpass('APIKEY: ')

In [7]:
%%writefile comment_this_line_for_run.py
store=1000
l=[]
for i,ds in pf.iterrows():
    print(i,end='\r')
    try:
        r=requests.get(f'http://clustercien.udea.edu.co:8080/scienti/product?apikey={APIKEY}&model_year=2018&COD_RH={ds.COD_RH}&COD_PRODUCTO={ds.COD_PRODUCTO}')
    except:
        time.sleep(0.1)
        continue

    d=r.json()
    #Articles with DOI, group or SGL_CATEGORY
    if d.get('TXT_DOI') or d.get('group') or d.get('SGL_CATEGORY'):
        l.append(d)
        if i%store==0:
            print(i,end='\r')
            f=open('product_type_111.json','w')
            json.dump(l,f)
            f.close()            
    time.sleep(0.01)
f=open('product_type_111.json','w')
json.dump(l,f)
f.close()    

Overwriting comment_this_line_for_run.py


### Load articles with DOI, SGL_CATEGORY or Group

In [2]:
f=open('product_type_111.json','r')
l=json.load(f)
f.close()

In [3]:
l=pd.DataFrame(l)

In [4]:
l.shape

(43897, 70)

### Normalize DOIS

In [11]:
import re
def normalize_dois(s):
    ss=s #store NaN
    if isinstance(s,str):
        ss=s.strip().lower()
        ss=re.sub('\.$','',ss) #proper doi end
        ss=re.split('https*:\/\/.*doi\.org\/',ss)[-1].strip() #also works with oadoi!
        #Quality checks
        if (re.search('https*:\/\/',ss) #exclude Improper URL
            or ss.find('/')==-1 # Must contains at least one '\/', exclude: '-', '--','00','n.a.', etc.
            or re.search('^.\/.$',ss) #exclude 'N/A', 'n/a' etc
            ): 
            ss=None
        else:
            ss=ss.split(':')[-1].strip() #Proper doi split
            ss=re.split('^.*doi\.org\/',ss)[-1].strip() #also works with oadoi!
    return ss

assert normalize_dois('10.18273/RevMed')=='10.18273/revmed'
assert normalize_dois('doi:10.18273/revmed')=='10.18273/revmed'
assert normalize_dois('doi: 10.18273/RevMed')=='10.18273/revmed'
assert normalize_dois('10.3390/molecules23102662.')=='10.3390/molecules23102662'
assert normalize_dois('https://doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://dx.doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://oadoi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('dx.doi.org/10.1155/2017/1702506')=='10.1155/2017/1702506'
assert normalize_dois('http://www.udea.edu.co/10.1155/2017/1702506') is None
assert normalize_dois('N/A') is None
assert normalize_dois('-') is None
assert normalize_dois('') is None
assert isinstance(normalize_dois(float('NaN')),float) #return NaN
assert normalize_dois(['10.18273/revmed','10.3823/1406'])==['10.18273/revmed','10.3823/1406'] #ignore not str objects

In [12]:
l['TXT_DOI']=l['TXT_DOI'].apply(normalize_dois)

In [13]:
l['same_title']=l.TXT_NME_PROD.apply(lambda s: l[l.TXT_NME_PROD==s].shape[0] ) #slow

In [14]:
l['same_doi']=l.TXT_DOI.apply(lambda s: l[l.TXT_DOI==s].shape[0] ) #slow

### Filter repeated entries

* Sort by 'SGL_CATEGORIA' → 'COD_INST_AVALA' 
* Drop duplicates without subset (not in)
```python
  ['COD_RH','COD_PRODUCTO','DTA_CREACION','DTA_ACTUALIZACION','SEQ_PRODUCTO','DTA_VINCULACION']
```
  * Remove key,values inside :
    'details' → 'article' →
    ```python
    ['COD_RH','COD_PRODUCTO','DTA_CREACION','DTA_ACTUALIZACION']
    ```
    and convert `[journal]` to `str`
* Group by:
  1. 'COD_INST_AVALA'
  1. 'TXT_WEB_PRODUCTO'
  1. 'author_others'
  
Conclusions: Similar entries can be different in:
```python
 ['TXT_WEB_PRODUCTO','COD_INST_AVALA','author_others','details']
```
Best products have 'COD_INST_AVALA','author_others','details'→'keywords'

In [5]:
l.shape

(43897, 70)

In [16]:
def remove_details_fields(l,Type=['article','keywords'],
        remove=['COD_RH','COD_PRODUCTO','DTA_CREACION','DTA_ACTUALIZACION','COD_PALABRA_CLAVE']):
    ll=[]
    for d in l:
        dd={}
        lll=[]
        for key in d:
            if key in Type:
                for di in d[key]:
                    newdi=dict([(k,di[k]) for k in di.keys() if k not in remove ])
                    lll.append(newdi)
                dd[key]=lll
            else:
                dd[key]=d[key]
        ll.append(dd)
    return ll

Convert list ot str to apply `.drop_duplicates`

In [17]:
lists=['author','language','product_type','group','author_others','details','institution','project']
for k in lists:
    l[f'{k}_str']=l[k].astype(str)

`'details`' → special list with time stamps inside 

In [18]:
l['details_str']=l['details'].apply(remove_details_fields).astype(str)

Fields not used in search for duplicates

In [19]:
not_subset=['COD_RH','COD_PRODUCTO','DTA_CREACION','DTA_ACTUALIZACION','SEQ_PRODUCTO','DTA_VINCULACION'
           ]+lists

In [20]:
l.shape

(43897, 80)

In [21]:
n=100 #large enough
subset=[x for x in l.columns if x not in not_subset]
#NaN at the the end
l=l.sort_values(['DTA_AVAL_INST','TXT_DOI','group_str'])
kkk=l.drop_duplicates(subset=subset[:n]).drop( [x for x in l.columns if x.find('_str')>-1 ],axis='columns' )
kkk.shape

(43611, 72)

clean `l`

In [36]:
l=l.drop( [x for x in l.columns if x.find('_str')>-1 ],axis='columns' )

Acceptable number of duplicated entries! Move the problem to ETL

In [22]:
if n<=len(subset):
    print(subset[n-1])

#### Patological cases

In [23]:
doi=l.sort_values('same_doi',ascending=False)[['same_doi','TXT_DOI']].iloc[0].to_dict()
doi

{'same_doi': 32, 'TXT_DOI': '10.1093/mmy/myv019'}

In [24]:
kkkk=kkk[kkk['TXT_DOI'].fillna('')==doi['TXT_DOI']]
kkkk.shape

(5, 72)

Same author registered the product several time (maybe update the product information several times: Recheck with thesis to understand the system)

In [25]:
kkkk[['COD_RH','COD_PRODUCTO','TXT_DOI','TXT_WEB_PRODUCTO','COD_INST_AVALA','SGL_CATEGORIA','details']]

,COD_RH,COD_PRODUCTO,TXT_DOI,TXT_WEB_PRODUCTO,COD_INST_AVALA,SGL_CATEGORIA,details
25107,0000351598,90,10.1093/mmy/myv019,http://mmy.oxfordjournals.org/content/53/5/520...,007300000887,ART-ART_A1,"[{'article': [{'COD_RH': '0000351598', 'COD_PR..."
25080,0000351598,100,10.1093/mmy/myv019,http://mmy.oxfordjournals.org/content/53/5/520...,NaN,NaN,"[{'article': [{'COD_RH': '0000351598', 'COD_PR..."
25094,0000351598,77,10.1093/mmy/myv019,http://www.oxfordjournals.org/our_journals/mmy...,NaN,NaN,"[{'article': [{'COD_RH': '0000351598', 'COD_PR..."
25095,0000351598,78,10.1093/mmy/myv019,http://mmy.oxfordjournals.org/content/53/5/520...,NaN,NaN,"[{'article': [{'COD_RH': '0000351598', 'COD_PR..."
25106,0000351598,89,10.1093/mmy/myv019,http://www.oxfordjournals.org/our_journals/mmy...,NaN,NaN,"[{'article': [{'COD_RH': '0000351598', 'COD_PR..."


Check equal titles

In [26]:
title=l.sort_values('same_title',ascending=False)[['same_title','TXT_NME_PROD']].drop_duplicates().iloc[1].to_dict() #0 is maximum same doi
title

{'same_title': 12,
 'TXT_NME_PROD': 'On intersubband absorption of radiation in delta-doped QWs'}

In [27]:
kkkk=kkk[kkk['TXT_NME_PROD'].fillna('')==title['TXT_NME_PROD']]
kkkk.shape

(5, 72)

Several authors registered the same product (DOI info normalized):

In [28]:
kkkk[['COD_RH','COD_PRODUCTO','TXT_DOI','TXT_WEB_PRODUCTO','COD_INST_AVALA','SGL_CATEGORIA','group']]

,COD_RH,COD_PRODUCTO,TXT_DOI,TXT_WEB_PRODUCTO,COD_INST_AVALA,SGL_CATEGORIA,group
32688,0000892076,10,10.1016/j.physe.2015.07.034,http://www.sciencedirect.com/science/article/p...,626800000880,ART-ART_A2,NaN
33574,0000971146,27,10.1016/j.physe.2015.07.034,www.elsevier.com/locate/physe,NaN,ART-ART_A2,NaN
33685,0000971146,36,10.1016/j.physe.2015.07.034,www.elsevier.com/locate/physe,NaN,ART-ART_A2,NaN
42439,0001532231,13,10.1016/j.physe.2015.07.034,http://www.sciencedirect.com/science/article/p...,NaN,ART-ART_A2,NaN
43142,0001605958,31,10.1016/j.physe.2015.07.034,NaN,NaN,ART-ART_A2,NaN


## Conclusions
* The level of really duplicated entries with the same relevant metadata seem to be rather low. The may be generated manually in some patological cases. We still need to check with other products like thesis check if an update of the product generate a new `'COD_PRODUCT0'` for the old product (see doi previous table: same product, same `'COD_RH'` but differents `'COD_PRODUCT0'`
* The information of same title or doi entries seem to be complementary:  group, aval, categories, keywords full authors info, etc
* Generated tools that could be implemented in ETL

### Find articles with associated projects

In [29]:
l[~l.project.isna()].shape

(9014, 80)

### Find CATEGORIAS of articles

In [30]:
l[~l['CAT_MED'].isna()].shape

(24702, 80)

In [31]:
l[~l['SGL_CATEGORIA'].isna()].shape

(35917, 80)

In [32]:
l['CAT_MED'].dropna().unique()

array(['ART-ART_D', 'ART-GC_ART', 'ART-ART_C', 'ART-ART_B', 'ART-ART_A2',
       'ART-ART_A1', 'ART-00'], dtype=object)

In [33]:
afadfad #Emergency stop

NameError: name 'afadfad' is not defined

Personal data

In [10]:
l['NRO_DOCUMENTO_IDENT']=l.author.str[0].str.get('NRO_DOCUMENTO_IDENT')

In [14]:
l=l.drop_duplicates(subset=['NRO_DOCUMENTO_IDENT']).reset_index(drop=True)

In [26]:
import getpass

In [27]:
ndi=getpass.getpass('NRO DOCUMENTO IDENTIDAD?')

NRO DOCUMENTO IDENTIDAD? ········


In [28]:
k=l[l['NRO_DOCUMENTO_IDENT']==ndi]

In [29]:
from IPython.display import JSON

In [31]:
#JSON(k.author.str[0].iloc[0])

In [17]:
l.author.str[0].iloc[0].keys()

dict_keys(['COD_RH', 'NRO_ID_CNPQ', 'TXT_NAMES_RH', 'TXT_PRIM_APELL', 'TXT_SEG_APELL', 'TXT_TOTAL_NAMES', 'TPO_NACIONALIDAD', 'DTA_NACIM', 'TPO_DOCUMENTO_IDENT', 'TPO_SEXO', 'COD_RH_MUN_NACIM', 'COD_MUN_NACIM', 'NRO_DOCUMENTO_IDENT', 'COD_RH_MUN_EXPED_DOC', 'COD_MUN_EXPED_DOC', 'COD_RH_RELIGION', 'SGL_PAIS_NACIM', 'TPO_ESTADO_CIVIL', 'TPO_DIRECCION_USUAL', 'TXT_CITACION_BIBLIO', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'STA_VISIBLE', 'TPO_PERFIL', 'TXT_CONF_DATOS', 'COD_TRAY_PROF_ACT', 'STA_ACTIVO', 'city'])

In [33]:
keys=['COD_RH',  'TXT_TOTAL_NAMES', 'TPO_NACIONALIDAD', 'DTA_NACIM', 'TPO_DOCUMENTO_IDENT', 'TPO_SEXO',  
           'NRO_DOCUMENTO_IDENT', 'COD_RH_RELIGION', 'SGL_PAIS_NACIM', 'TPO_ESTADO_CIVIL',
           'COD_ORCID','AUTOR_ID_SCP' ]
for  k in keys:
    l[k]=l.author.str[0].str.get(k)

In [35]:
l[keys]

,COD_RH,TXT_TOTAL_NAMES,TPO_NACIONALIDAD,DTA_NACIM,TPO_DOCUMENTO_IDENT,TPO_SEXO,NRO_DOCUMENTO_IDENT,COD_RH_RELIGION,SGL_PAIS_NACIM,TPO_ESTADO_CIVIL,COD_ORCID,AUTOR_ID_SCP
0,0000000011,NATALIA ANDREA ROJAS HENAO,C,"Sun, 21 Dec 1986 00:00:00 GMT",C,F,1039446856,0000000000,COL,S,None,None
1,0000000113,Jhon Jairo Rodriguez,C,"Sun, 18 Dec 1977 00:00:00 GMT",C,M,98395448,0000000000,COL,S,None,None
2,0000000132,Jessica Paola Rey Suarez,C,"Sun, 08 May 1988 00:00:00 GMT",C,F,1110473903,0000000000,COL,C,0000-0003-3299-2246,1110473903
3,0000000147,Doris Margarita Correa Rios,C,"Wed, 24 Nov 1965 00:00:00 GMT",C,F,21852251,0000000000,COL,S,None,None
4,0000000153,Lina Maria Preciado Rojo,C,"Tue, 24 Feb 1987 00:00:00 GMT",C,F,1017148632,0000000000,COL,S,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4891,0001771850,JUAN PAULO RUIZ TABORDA,C,"Thu, 22 Nov 1990 00:00:00 GMT",C,M,1039454230,None,COL,S,None,None
4892,0001778935,Juan Felipe Varela Garcia,C,"Sat, 08 Jan 1994 00:00:00 GMT",C,M,1036650019,None,COL,S,None,None
4893,0001781054,Karen Gomez David,C,"Mon, 07 Mar 1994 00:00:00 GMT",C,F,1037632533,None,COL,S,0000-0002-3051-100X,None
4894,0001787019,Priscila Cardoso Calegari,B,"Thu, 03 Jun 1982 00:00:00 GMT",P,F,FS564641,None,BRA,C,None,None


## Appendix
Understand the data

In [ ]:
qq=pd.DataFrame( kkk.iloc[1:]['details'].str[0].str['article'].str[0].to_list())
qq['journal']=qq['journal'].astype(str)
qq[[k for k in qq.columns if k not in ['COD_RH', 'COD_PRODUCTO',
                    'DTA_CREACION','DTA_ACTUALIZACION'] ]].drop_duplicates()

In [ ]:
diffs=['COD_RH','COD_PRODUCTO','SGL_CATEGORIA','TXT_WEB_PRODUCTO','DTA_CREACION',
       'DTA_ACTUALIZACION','DTA_AVAL_INST','TPO_AVAL_INST','COD_INST_AVALA',
       'ID_USUARIO_AVAL','SEQ_PRODUCTO','DTA_VINCULACION','STA_MEDIDO',
       'author_others','details','institution']
#Extract product with TPO_AVAL_INST
DIFFS=['SGL_CATEGORIA','TXT_WEB_PRODUCTO','DTA_AVAL_INST','TPO_AVAL_INST','COD_INST_AVALA',
       'ID_USUARIO_AVAL','SEQ_PRODUCTO','DTA_VINCULACION','STA_MEDIDO',
       'author_others','details','institution']
lists=['author','language','product_type','group']

Fill the previous lists

In [ ]:
kkk=kk[[k for k in kk.columns if k not in diffs]].reset_index(drop=True)

In [ ]:
kkkk=kkk[list(kkk.columns)+lists].reset_index(drop=True)

In [ ]:
for k in lists:
    kkkk[k]=kkkk[k].astype(str)
kkkk.drop_duplicates()

Best

In [ ]:
kkk=kk[DIFFS].sort_values(['SGL_CATEGORIA','TXT_WEB_PRODUCTO'])
BEST=kkk.iloc[[0]]
BEST

Others

In [ ]:
WORST=kkk.iloc[1:].drop_duplicates(['DTA_AVAL_INST','TPO_AVAL_INST','COD_INST_AVALA',
                                    'ID_USUARIO_AVAL','STA_MEDIDO'])
WORST

In [ ]:
qq=pd.DataFrame( kkk.iloc[1:]['details'].str[0].str['article'].str[0].to_list())
qq['journal']=qq['journal'].astype(str)
qq[[k for k in qq.columns if k not in ['COD_RH', 'COD_PRODUCTO',
                    'DTA_CREACION','DTA_ACTUALIZACION'] ]].drop_duplicates()

In [ ]:
kkk.iloc[1:]['institution'].astype(str).drop_duplicates()

In [ ]:
kkk.iloc[1:].drop_duplicates(['TXT_WEB_PRODUCTO'])['TXT_WEB_PRODUCTO']

In [ ]:
kkk.iloc[1:]['author_others'].astype(str).drop_duplicates()

In [ ]:
kkk.iloc[1:][kkk.iloc[1:]['author_others'].apply(len)>1]

Repeated

In [ ]:
rr=kkk.iloc[1:]
rr=rr.sort_values('TXT_WEB_PRODUCTO')[:19]
rr['author_others'].astype(str).drop_duplicates()

In [ ]:
rr[:1]

In [ ]:
rr2=kkk.iloc[1:]
rr2=rr2.sort_values('TXT_WEB_PRODUCTO')[19:]
print(rr2.shape[0])
rr2=rr2[rr2['author_others'].apply(len)==1]
rr2

In [ ]:
pd.DataFrame( kk['group'].str[0].to_list() )[
    ['NRO_ID_GRUPO', 'COD_ID_GRUPO', 'NME_GRUPO', 'ANO_FORMACAO',
       'MES_FORMACAO', 'COD_RH_AREA', 'COD_AREA_CONHEC', 'COD_PROGRAMA',
       'COD_PROGRAMA_SECUND', 'TPO_ESTADO_GR', 'TXT_CLASIF', 'DTA_CLASIF',
       'DTA_FIN_CLASIF', 'TPO_GRUPO', 'DTA_TPO_GRUPO', 'STA_ELIMINADO',
       'TXT_PLAN_TRABAJO', 'TXT_ESTADO_ARTE', 'TXT_OBJETIVOS', 'TXT_RETOS',
       'TXT_VISION', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'DTA_CREACION',
       'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'DTA_INI_SESION',
       'TXT_CLASIF_1', 'COD_GRAN_AREA_CONHEC_MED', 'STA_CENTRO', 'TPO_CENTRO',
       'STA_SECTOR', 'NRO_INTENTOS', 'STA_ACTIVO', 'COD_AREA_CONHEC_BK']].drop_duplicates()

In [ ]:
kk.shape

In [ ]:
kk.drop_duplicates().shape

In [ ]:
pic=l[l['SGL_CATEGORIA'].fillna('').str.contains('PIC')]

In [ ]:
from IPython import display

In [ ]:
#display.JSON(pic.iloc[0].dropna().to_dict())

In [41]:
[k for k in l.columns if re.search('[a-z]',k) ]

['author',
 'language',
 'product_type',
 'author_others',
 'details',
 'institution',
 'group',
 'project',
 'same_title',
 'same_doi']